# Train_test Results

## Connection to GoogleDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Libraries installation

Exécutez les cellules suivantes pour installer les librairies :

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.0 MB/s eta 0:00:00


In [13]:
!pip install tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 KB 6.9 MB/s eta 0:00:00


In [5]:
import torch, os
import datasets
import numpy as np
from torch.utils.data import DataLoader
from transformers import BertTokenizer,BertForSequenceClassification,AutoModelForSequenceClassification, AdamW
from progressbar import progressbar
from tqdm import tqdm

## Tokenizer m-BERT

Nous implémentons le tokenizer m-BERT :

In [19]:
def encode_text_dataset(dataset, model_name, output_dir):
    # Charger le modèle mBERT pré-entraîné
    tokenizer = BertTokenizer.from_pretrained(model_name)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Encoder le texte dans le jeu de données
    for i in progressbar(range(len(dataset))):
        text = dataset[i]
        encoded_text = tokenizer(text, return_tensors='pt')
        output_file = os.path.join(output_dir, f"encoded_text_{i}.pt")
        torch.save(encoded_text, output_file)

    # Vider le cache de mémoire PyTorch
    torch.cuda.empty_cache()

In [20]:
def tok_dataset(dataset,tokenizer):
    toked = dataset.map(tok)
    dic = {}
    dic['input_ids']= toked['input_ids']
    dic['attention_mask']= toked['attention_mask']
    dic['token_type_ids']= toked['token_type_ids']
    dic['labels'] = [x for x in dataset['Label']]#[ [1 if j == x else 0 for j in range(num_labels)]for x in dataset['Label']]
    output  = datasets.Dataset.from_dict(dic).with_format("torch")
    return output

## Model m-BERT

Voici les hyperparamètres pour notre modèle :

In [21]:
#Hyperparameters
MAX_LENGTH = 50
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
EPOCH = 1

In [22]:
def tok(examples):
    return tokenizer(examples['Utterance'], padding="max_length", max_length=MAX_LENGTH,truncation=True)



---

Exécuter la cellule suivante qui est notre fonction pour l'exécution du training:

In [24]:
def exec_train(model,tokenizer,dataset):
    #use device
    device  = torch.device('cuda') if torch.cuda.is_available() else torch.device
    model.to(device)
    tok_train = tok_dataset(dataset,tokenizer)
    data_train = DataLoader(tok_train,batch_size=BATCH_SIZE,shuffle=True)
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    #Training
    for epoch in range(EPOCH):
        model.train()
        pbar = tqdm(total= len(data_train),desc=f'Training epoch {epoch+1}')
        nb_samples = 0
        loss_tot = 0
        total_correct = 0
        for batch in data_train:
            optimizer.zero_grad()
            batch['input_ids'] = batch['input_ids'].to(device)
            batch['attention_mask'] = batch['attention_mask'].to(device)
            batch['token_type_ids'] = batch['token_type_ids'].to(device)
            batch['labels'] = batch['labels'].long().to(device)
            outputs = model(**batch)
            nb_samples += BATCH_SIZE
            
            #loss
            loss = outputs.loss
            loss_tot += float(loss)
            
            #accuracy
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            total_correct += (predictions == batch['labels']).sum().item()
            
            #prompt
            pbar.set_postfix(loss=loss_tot/nb_samples,accuracy=total_correct/nb_samples)
            
            #learning
            loss.backward()
            optimizer.step()
            
            pbar.update(1)
    #return model



---


Exécuter la cellule suivante qui est notre fonction pour l'exécution du test :

In [26]:
def exec_test(model,tokenizer,dataset):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    tok_test = tok_dataset(dataset,tokenizer)
    data_test = DataLoader(tok_test,batch_size=BATCH_SIZE,shuffle=False)
    #validation
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        pbar = tqdm(total= len(data_test),desc='Test')
        for batch in data_test:
            batch['input_ids'] = batch['input_ids'].to(device)
            batch['attention_mask'] = batch['attention_mask'].to(device)
            batch['token_type_ids'] = batch['token_type_ids'].to(device)
            batch['labels'] = batch['labels'].long().to(device)
            
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            total_correct += (predictions == batch['labels']).sum().item()
            total_samples += batch['labels'].size(0)
            pbar.set_postfix(accuracy=total_correct/total_samples)
            pbar.update(1)
        val_acc = total_correct / total_samples
        print('test Acc:', val_acc)

## Entraînement du modèle sur les différents datasets de [miam](https://huggingface.co/datasets/miam)

Voici l'ensemble des datasets avec leur nombre d'uttérances que l'on peut retrouver sur [Hugging Face](https://huggingface.co/datasets/miam) :

| Dataset name          | Language                                             | Train                    | Valid                    | Test                    |
|--------------------------|----------------------------------------------------|--------------------------|--------------------------|-------------------------|
| dihana                   | Spanish                                           | 19063                    | 2123                     |2361                     |     
| ilisten                  | Italian                                             | 1986                     | 230                      |971                      |    
| loria                    | French                                           | 8465                     | 942                      |1047                     |    
| maptask                  | English                                            | 25382                    | 5221                     |5335           |             
| vm2                      | German                                           | 25060                    | 2860                     |2855   |         


### Dataset `dihana`

In [35]:
#Load Dataset
miam= '/content/drive/MyDrive/ANLP/projet_anlp/miam.py'
dataset = datasets.load_dataset('miam','dihana')

Generating train split:   0%|          | 0/19063 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2123 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2361 [00:00<?, ? examples/s]

Dataset miam downloaded and prepared to /root/.cache/huggingface/datasets/miam/dihana/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
num_labels = len(set(dataset['train']['Label']))

In [37]:
#Import model et tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', return_tensors='pt')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=num_labels, problem_type="single_label_classification")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [38]:
exec_train(model,tokenizer,datasets.concatenate_datasets([dataset['train'],dataset['validation']]))

Map:   0%|          | 0/21186 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training epoch 1: 100%|██████████| 663/663 [03:16<00:00,  3.38it/s, accuracy=0.909, loss=0.00893]


In [39]:
exec_test(model,tokenizer,dataset['test'])

Map:   0%|          | 0/2361 [00:00<?, ? examples/s]

Test: 100%|██████████| 74/74 [00:06<00:00, 11.33it/s, accuracy=0.916]

test Acc: 0.9161372299872935


### Dataset `ilisten`

In [40]:
#Load Dataset
miam= '/content/drive/MyDrive/ANLP/projet_anlp/miam.py'
dataset = datasets.load_dataset('miam','ilisten')

Generating train split:   0%|          | 0/1986 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/230 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/971 [00:00<?, ? examples/s]

Dataset miam downloaded and prepared to /root/.cache/huggingface/datasets/miam/ilisten/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
num_labels = len(set(dataset['train']['Label']))

In [42]:
#Import model et tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', return_tensors='pt')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=num_labels, problem_type="single_label_classification")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [43]:
exec_train(model,tokenizer,datasets.concatenate_datasets([dataset['train'],dataset['validation']]))

Map:   0%|          | 0/2216 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training epoch 1: 100%|██████████| 70/70 [00:20<00:00,  3.36it/s, accuracy=0.529, loss=0.047]


In [44]:
exec_test(model,tokenizer,dataset['test'])

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

Test: 100%|██████████| 31/31 [00:02<00:00, 10.61it/s, accuracy=0.708]

test Acc: 0.7075180226570545


### Dataset `loria`

In [29]:
#Load Dataset
miam= '/content/drive/MyDrive/ANLP/projet_anlp/miam.py'
dataset = datasets.load_dataset('miam','loria')

Generating train split:   0%|          | 0/8465 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/942 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1047 [00:00<?, ? examples/s]

Dataset miam downloaded and prepared to /root/.cache/huggingface/datasets/miam/loria/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [30]:
num_labels = len(set(dataset['train']['Label']))

In [31]:
#Import model et tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', return_tensors='pt')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=num_labels, problem_type="single_label_classification")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [32]:
exec_train(model,tokenizer,datasets.concatenate_datasets([dataset['train'],dataset['validation']]))

Map:   0%|          | 0/9407 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training epoch 1: 100%|██████████| 294/294 [01:27<00:00,  3.37it/s, accuracy=0.693, loss=0.0371]


In [33]:
exec_test(model,tokenizer,dataset['test'])

Map:   0%|          | 0/1047 [00:00<?, ? examples/s]

Test: 100%|██████████| 33/33 [00:02<00:00, 11.22it/s, accuracy=0.771]

test Acc: 0.7707736389684814


### Dataset `maptask`

In [55]:
#Load Dataset
miam= '/content/drive/MyDrive/ANLP/projet_anlp/miam.py'
dataset = datasets.load_dataset('miam','maptask')

  0%|          | 0/3 [00:00<?, ?it/s]

In [46]:
num_labels = len(set(dataset['train']['Label']))

In [47]:
#Import model et tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', return_tensors='pt')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=num_labels, problem_type="single_label_classification")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [48]:
exec_train(model,tokenizer,datasets.concatenate_datasets([dataset['train'],dataset['validation']]))

Map:   0%|          | 0/30603 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training epoch 1: 100%|██████████| 957/957 [04:42<00:00,  3.39it/s, accuracy=0.565, loss=0.0402]


In [49]:
exec_test(model,tokenizer,dataset['test'])

Map:   0%|          | 0/5335 [00:00<?, ? examples/s]

Test: 100%|██████████| 167/167 [00:14<00:00, 11.34it/s, accuracy=0.616]

test Acc: 0.6164948453608248


### Dataset `vm2`

In [50]:
#Load Dataset
miam= '/content/drive/MyDrive/ANLP/projet_anlp/miam.py'
dataset = datasets.load_dataset('miam','vm2')

Generating train split:   0%|          | 0/25060 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2860 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2855 [00:00<?, ? examples/s]

Dataset miam downloaded and prepared to /root/.cache/huggingface/datasets/miam/vm2/1.0.0/3cb25c5337f9e60db1dc6d90344763a6ef79d7a4ac3c5f215ce6e8afe99db26c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [51]:
num_labels = len(set(dataset['train']['Label']))

In [52]:
#Import model et tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', return_tensors='pt')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=num_labels, problem_type="single_label_classification")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [53]:
exec_train(model,tokenizer,datasets.concatenate_datasets([dataset['train'],dataset['validation']]))

Map:   0%|          | 0/27920 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training epoch 1: 100%|██████████| 873/873 [04:18<00:00,  3.38it/s, accuracy=0.498, loss=0.0536]


In [54]:
exec_test(model,tokenizer,dataset['test'])

Map:   0%|          | 0/2855 [00:00<?, ? examples/s]

Test: 100%|██████████| 90/90 [00:07<00:00, 11.33it/s, accuracy=0.533]

test Acc: 0.5330998248686515


## Résultats

Voici les différents résultats obtenus pour chaque dataset avec le modèle m-BERT multilingue. Nous utilisons ici la métrique `accuracy` (en %) :


| Dataset name          | Language                                             | Train & Valid                 | Test                    |
|--------------------------|----------------------------------------------------|--------------------------|--------------------------|
| dihana                   | Spanish                                           | 90.9                                   |91.6                     |     
| ilisten                  | Italian                                             | 52.9                     | 70.8                     |    
| loria                    | French                                           | 69.3                             |77.1                     |    
| maptask                  | English                                            | 56.5                             |61.6           |             
| vm2                      | German                                           | 49.8                           |53.3   |   
|--------------------------|----------------------------------------------------|--------------------------|--------------------------|
| **Total**                      |                                            | **63.9**                       |**70.9**   |   
      
   


Nous faisons uniquement une `epoch` pour l'entraînement de notre modèle, ce qui permet d'éviter l'overfitting. Dans ce cas-ci le `valid set` n'est pas nécessaire et nous regroupons le `train` et `valid` sets pour le training.



---
Si nous comparons nos résultats à celui du papier original [Code-switched inspired losses for spoken dialog representations](https://aclanthology.org/2021.emnlp-main.656.pdf), nous retrouvons des résultats similaires, avec une moyenne de **70.9% d'accuracy** pour notre modèle **vs.** **73.8%** pour le modèle du papier original.
